### Calls our final API with a sample of our trainings data and generates metricks

In [10]:
import os
import requests
import pandas as pd
from PIL import Image
from datetime import datetime
import io
import random
# Initialize an empty dataframe
df = pd.DataFrame(columns=['labels','pred_labels', 'con_nowildfire', 'con_wildfire', 'start_time', 'end_time'])

# Directories
base_dir = '../../data/test'
directories = [os.path.join(base_dir, d) for d in ['nowildfire', 'wildfire']]

for directory in directories:
    sample = random.sample(os.listdir(directory), 500) # select 500 images as samples for each directory
    for filename in sample:
        if filename.endswith(".jpg"):
            file_path = os.path.join(directory, filename)
            # Open the image
            img = Image.open(file_path)
            img_byte_arr = io.BytesIO()
            img.save(img_byte_arr, format='TIFF')
            img_byte_arr = img_byte_arr.getvalue()        
            # API call
            headers = {'Content-Type': 'image/tiff'}
            url = 'https://staging-yatai.yatai.k8s.eai.dziubalabs.de/predict_image' 
            
            start_time = datetime.now() # timestamp before API call
            
            # Send image
            response = requests.post(url,data=img_byte_arr, headers=headers)
            
            end_time = datetime.now() # timestamp after API call
            
            # Parse the response
            result = response.json()
            
            # Prepare the label
            label = 0 if 'nowildfire' in directory else 1
            pred_label = 0 if result[0] > result[1] else 1
            # Append the result to the dataframe
            new_df = pd.DataFrame([{
                'labels': label,
                'pred_labels' : pred_label,
                'con_nowildfire': result[0], 
                'con_wildfire': result[1], 
                'start_time': start_time, 
                'end_time': end_time
            }])
            df = pd.concat([df, new_df], axis=0, ignore_index=True)
            if len(df)%10 == 0:
                print(len(df))

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000


In [11]:
df['duration'] = df['end_time'] - df['start_time']

In [46]:
df['duration'] = df['duration'].apply(pd.to_timedelta)

In [12]:
df.to_csv('sample_test_inference.csv')

In [48]:
df

,Unnamed: 0,labels,pred_labels,con_nowildfire,con_wildfire,start_time,end_time,duration
0,0,0,0,0.949862,0.050138,2023-07-17 21:25:50.460500,2023-07-17 21:25:51.368634,0 days 00:00:00.908134
1,1,0,1,0.999988,0.000012,2023-07-17 21:25:51.373815,2023-07-17 21:25:52.550183,0 days 00:00:01.176368
2,2,0,0,0.960480,0.039520,2023-07-17 21:25:52.553710,2023-07-17 21:25:53.427595,0 days 00:00:00.873885
3,3,0,1,0.999992,0.000007,2023-07-17 21:25:53.431562,2023-07-17 21:25:54.356054,0 days 00:00:00.924492
4,4,0,1,0.248964,0.751036,2023-07-17 21:25:54.359307,2023-07-17 21:25:55.232226,0 days 00:00:00.872919
...,...,...,...,...,...,...,...,...
995,995,1,1,0.000591,0.999409,2023-07-17 22:09:44.765737,2023-07-17 22:09:45.865704,0 days 00:00:01.099967
996,996,1,1,0.001756,0.998244,2023-07-17 22:09:45.867576,2023-07-17 22:09:46.723938,0 days 00:00:00.856362
997,997,1,1,0.001432,0.998568,2023-07-17 22:09:46.726451,2023-07-17 22:09:47.735307,0 days 00:00:01.008856
998,998,1,1,0.001454,0.998546,2023-07-17 22:09:47.737301,2023-07-17 22:09:49.057795,0 days 00:00:01.320494


In [22]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [47]:
accuracy = accuracy_score(df['labels'], df['pred_labels'])
recall = recall_score(df['labels'], df['pred_labels'], average='macro')
precision = precision_score(df['labels'], df['pred_labels'], average='macro')
f1 = f1_score(df['labels'], df['pred_labels'], average='macro')
avg_inference = df['duration'].mean()
print('Accuracy: ', accuracy)
print('Recall: ', recall)
print('Precision: ', precision)
print('F1 Score: ', f1)
print('Average inference time ', avg_inference)

Accuracy:  0.738
Recall:  0.738
Precision:  0.792607600912739
F1 Score:  0.7251779001074115
Average inference time  0 days 00:00:02.636476259


The accuracy seem low but we sampled only 1000 of our 6000 images in the test set in order to get a feeling on how the server handles load. The model overall scores a test accuracy of 91% on the whole test dataset.